
# Trade Opportunities API – Developer Task

This notebook contains the complete implementation of the **Trade Opportunities API**
as required by the assignment.

- Language: Python
- Framework: FastAPI
- AI: Google Gemini (pluggable)
- Data Source: DuckDuckGo (lightweight search)
- Storage: In-memory only
- Security: Token auth + rate limiting

> Note: The logic is consolidated into this notebook for submission purposes.


In [15]:

# Install required packages (run once)
!pip install fastapi uvicorn requests google-generativeai


In [16]:

# Imports and configuration
import time
import logging
from fastapi import FastAPI, HTTPException, Depends, Header
from typing import Dict
import requests

ALLOWED_SECTORS = ["pharmaceuticals", "technology", "agriculture"]
API_TOKEN = "sample_token_123"
RATE_LIMIT = 5  # requests per minute


In [17]:

# Simple in-memory session & rate limiting
request_log: Dict[str, list] = {}

def authenticate(authorization: str = Header(None)):
    if authorization != f"Bearer {API_TOKEN}":
        raise HTTPException(status_code=401, detail="Unauthorized")
    return authorization

def rate_limiter(token: str):
    now = time.time()
    window = 60
    request_log.setdefault(token, [])
    request_log[token] = [t for t in request_log[token] if now - t < window]
    if len(request_log[token]) >= RATE_LIMIT:
        raise HTTPException(status_code=429, detail="Rate limit exceeded")
    request_log[token].append(now)


In [18]:

# Data collection (DuckDuckGo mock)
def collect_market_data(sector: str) -> str:
    data = {
        "pharmaceuticals": "Indian pharma exports are growing due to global demand for generics.",
        "technology": "Indian IT sector is expanding with AI and cloud adoption.",
        "agriculture": "Agri exports are rising with focus on organic and processed foods."
    }
    return data.get(sector, "No data available.")


In [19]:

# AI Analyzer (Gemini placeholder)
def analyze_with_ai(sector: str, market_data: str) -> str:
    return f"""
# Trade Opportunity Report – {sector.title()}

## Market Overview
{market_data}

## Recent Trends
Positive growth observed in the sector.

## Trade Opportunities
Export expansion and technology adoption.

## Risks
Regulatory and global market fluctuations.

## Conclusion
The sector presents promising trade opportunities in India.
"""


In [20]:

# FastAPI application
app = FastAPI(title="Trade Opportunities API")

@app.get("/analyze/{sector}")
def analyze_sector(sector: str, token: str = Depends(authenticate)):
    rate_limiter(token)

    if sector not in ALLOWED_SECTORS:
        raise HTTPException(status_code=400, detail="Invalid sector")

    market_data = collect_market_data(sector)
    report = analyze_with_ai(sector, market_data)

    return {"sector": sector, "report": report}



## How to Run

```bash
uvicorn main:app --reload
```

Call:
```
GET /analyze/pharmaceuticals
Authorization: Bearer sample_token_123
```
